In [1]:
import numpy as np
import multiprocessing as mp
from functions import weightsample, als, rk, alstest, rktest, alswrite, rkwrite, listener, alsmp, rkmp, read

# Sparse Tests

## 50x50 matrix

In [2]:
sfactor50 = np.random.choice(4, size=(50,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
sweight50 = np.random.randint(0, 2, size=(4, 50))
s50 = np.matmul(sfactor50, sweight50)
np.count_nonzero(s50 == 0) / (s50.shape[0]*s50.shape[1])

0.5808

In [7]:
%%time
fname = "Errors/salserr50.txt"
alsmp(s50, k = 4, niter = 100, filename = fname, loop = 15)
salserr50 = np.asarray(read(fname)[:-1]).astype(float)
print(salserr50)

[0.38420618 0.29623185 0.42163281 0.40645825 0.37964073 0.41054305
 0.44203489 0.30687028 0.31597839 0.35182171 0.35256397 0.34254329
 0.31229496 0.4081136  0.40742362]
Wall time: 821 ms


In [8]:
%%time
fname = "Errors/srkerr50.txt"
rkmp(s50, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
srkerr50 = np.asarray(read(fname)[:-1]).astype(float)
print(srkerr50)

[1.18390955 0.98944136 1.05139001 1.12116848 1.13755777 1.10441665
 0.97488825 1.1612159  0.88377034 1.14643438 0.98412324 1.11999015
 1.11724917 1.24702333 0.98698378]
Wall time: 1min 14s


## 200x200 matrix

In [2]:
sfactor200 = np.random.choice(4, size=(200,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
sweight200 = np.random.randint(0, 2, size=(4, 200))
s200 = np.matmul(sfactor200, sweight200)
np.count_nonzero(s200 == 0) / (s200.shape[0]*s200.shape[1])

0.4783

In [3]:
%%time
fname = "Errors/salserr200.txt"
alsmp(s200, k = 4, niter = 100, filename = fname, loop = 15)
salserr200 = np.asarray(read(fname)[:-1]).astype(float)
print(salserr200)

[0.71103769 0.69197473 0.67494066 0.69020619 0.68115356 0.70798929
 0.67101343 0.69120031 0.7113065  0.73743839 0.69970802 0.68761777
 0.72392372 0.68134634 0.71033734]
Wall time: 28.6 s


In [4]:
%%time
fname = "Errors/srkerr200.txt"
rkmp(s200, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
srkerr200 = np.asarray(read(fname)[:-1]).astype(float)
print(srkerr200)

[1.0848787  1.0375964  1.08695894 0.99857039 1.00148092 0.94731999
 0.98861656 0.97466809 1.06371218 0.95769236 0.94243534 0.98939098
 0.99609502 0.92532257 0.94898401]
Wall time: 2min 7s


## 1000x1000 matrix

In [2]:
sfactor1k = np.random.choice(4, size=(1000,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
sweight1k = np.random.randint(0, 2, size=(4, 1000))
s1k = np.matmul(sfactor1k, sweight1k)
np.count_nonzero(s1k == 0) / (s1k.shape[0]*s1k.shape[1])

0.5227

In [3]:
%%time
fname = "Errors/salserr1k.txt"
alsmp(s1k, k = 4, niter = 100, filename = fname, loop = 15)
salserr1k = np.asarray(read(fname)[:-1]).astype(float)
print(salserr1k)

[0.80573552 0.80216327 0.80657376 0.81001677 0.80149177 0.79916573
 0.80612909 0.80201629 0.80344161 0.8027064  0.80175479 0.81126418
 0.80535349 0.80105497 0.81371224]
Wall time: 10.6 s


In [7]:
%%time
fname = "Errors/srkerr1k.txt"
rkmp(s1k, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
srkerr1k = np.asarray(read(fname)[:-1]).astype(float)
print(srkerr1k)

[0.86994533 0.89335083 0.89514653 0.86829225 0.87297784 0.88015126
 0.85873812 0.85388062 0.87639941 0.8800276  0.85929324 0.88330288
 0.89778083 0.88504959 0.86998381]
Wall time: 5min 32s


## 10,000x10,000 matrix

In [4]:
sfactor10k = np.random.choice(4, size=(10000,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
sweight10k = np.random.randint(0, 2, size=(4, 10000))
s10k = np.matmul(sfactor10k, sweight10k)
np.count_nonzero(s10k == 0) / (s10k.shape[0]*s10k.shape[1])

0.5212851

In [5]:
%%time
fname = "Errors/salserr10k.txt"
alsmp(s10k, k = 4, niter = 100, filename = fname, loop = 15, cores = 4)
salserr10k = np.asarray(read(fname)[:-1]).astype(float)
print(salserr10k)

[0.82840739 0.82504429 0.82773252 0.82928556 0.82763171 0.8285348
 0.82737003 0.82887042 0.82889542 0.82897594 0.82679751 0.82827703
 0.82912263 0.82773023 0.82671306]
Wall time: 15min 20s


In [5]:
%%time
fname = "Errors/srkerr1k.txt"
rkmp(s10k, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
srkerr10k = np.asarray(read(fname)[:-1]).astype(float)
print(srkerr10k)

OSError: [WinError 1450] Insufficient system resources exist to complete the requested service

# Dense Tests

## 50x50 matrix

In [6]:
dfactor50 = np.random.choice(4, size=(50,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
dweight50 = np.random.randint(0, 2, size=(4, 50))
d50 = np.matmul(dfactor50, dweight50)
np.count_nonzero(d50 == 0) / (d50.shape[0]*d50.shape[1])

0.4752

In [7]:
%%time
fname = "Errors/dalserr50.txt"
alsmp(d50, k = 4, niter = 100, filename = fname, loop = 15)
dalserr50 = np.asarray(read(fname)[:-1]).astype(float)
print(dalserr50)

[0.38263496 0.35401066 0.27905093 0.32620063 0.39569974 0.39578188
 0.36409114 0.3048924  0.34788133 0.29279417 0.26049527 0.36268124
 0.21318269 0.28093075 0.46004767]
Wall time: 1.82 s


In [8]:
%%time
fname = "Errors/drkerr50.txt"
rkmp(d50, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
drkerr50 = np.asarray(read(fname)[:-1]).astype(float)
print(drkerr50)

[0.94020891 0.98859558 1.09634754 0.81194547 1.02277047 0.87629311
 0.92480319 0.83381118 0.90112286 1.08505669 0.94675363 0.84551977
 1.06286931 0.89287942 0.92097431]
Wall time: 1min 41s


## 200x200 matrix

In [9]:
dfactor200 = np.random.choice(4, size=(200,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
dweight200 = np.random.randint(0, 2, size=(4, 200))
d200 = np.matmul(dfactor200, dweight200)
np.count_nonzero(d200 == 0) / (d200.shape[0]*d200.shape[1])

0.557975

In [10]:
%%time
fname = "Errors/dalserr200.txt"
alsmp(d200, k = 4, niter = 100, filename = fname, loop = 15)
dalserr200 = np.asarray(read(fname)[:-1]).astype(float)
print(dalserr200)

[0.7505875  0.73609073 0.73429381 0.73365637 0.72740865 0.73193863
 0.74160516 0.73112261 0.71255508 0.74260293 0.69377087 0.70675201
 0.73418556 0.72176739 0.7504969 ]
Wall time: 1min 31s


In [11]:
%%time
fname = "Errors/drkerr200.txt"
rkmp(s200, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
drkerr200 = np.asarray(read(fname)[:-1]).astype(float)
print(drkerr200)

NameError: name 's200' is not defined

## 1000x1000 matrix

In [12]:
dfactor1k = np.random.choice(4, size=(1000,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
dweight1k = np.random.randint(0, 2, size=(4, 1000))
d1k = np.matmul(dfactor1k, dweight1k)
np.count_nonzero(d1k == 0) / (d1k.shape[0]*d1k.shape[1])

0.515667

In [13]:
%%time
fname = "Errors/dalserr1k.txt"
alsmp(d1k, k = 4, niter = 100, filename = fname, loop = 15)
dalserr1k = np.asarray(read(fname)[:-1]).astype(float)
print(dalserr1k)

[0.80119571 0.80050625 0.80745006 0.80232816 0.80438557 0.79882916
 0.80373519 0.8007066  0.80663449 0.8023012  0.80767181 0.8024847
 0.80163144 0.80524146 0.80155247]
Wall time: 1min 41s


In [14]:
%%time
fname = "Errors/drkerr1k.txt"
rkmp(s1k, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
drkerr1k = np.asarray(read(fname)[:-1]).astype(float)
print(drkerr1k)

NameError: name 's1k' is not defined

## 10,000x10,000 matrix

In [15]:
dfactor10k = np.random.choice(4, size=(10000,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
dweight10k = np.random.randint(0, 2, size=(4, 10000))
d10k = np.matmul(dfactor10k, dweight10k)
np.count_nonzero(d10k == 0) / (d10k.shape[0]*d10k.shape[1])

0.5229483

In [16]:
%%time
fname = "Errors/dalserr10k.txt"
alsmp(d10k, k = 4, niter = 100, filename = fname, loop = 15)
dalserr10k = np.asarray(read(fname)[:-1]).astype(float)
print(salserr10k)

OSError: [WinError 1450] Insufficient system resources exist to complete the requested service

In [ ]:
%%time
fname = "Errors/drkerr1k.txt"
rkmp(s10k, k = 4, niter = 100, kacziter = 1000, filename = fname, loop = 15)
drkerr10k = np.asarray(read(fname)[:-1]).astype(float)
print(drkerr10k)